In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing the necessary libraries for the dataset which helps in EDA and Training and Testing the dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

Importing the training and testing dataset

In [ ]:
train_df=pd.read_csv('../input/titanic/train.csv')
test_df=pd.read_csv('../input/titanic/test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df

In [ ]:
print(train_df.shape)
print(test_df.shape)

    Merging both training and testing dataframe into single dataframe 'titanic_df'

In [ ]:
titanic_df=pd.concat([train_df,test_df],ignore_index=True)

In [ ]:
titanic_df[(titanic_df['Survived']!=0) & (titanic_df['Survived']!=1)]

In [ ]:
titanic_df.Survived.unique()

The Name and Ticket Number column is not required as it doesn't make any sense to dataframe let's drop those columns

In [ ]:
titanic_df.drop(['Name','Ticket'],axis=1,inplace=True)

Check for any NaN values

In [ ]:
titanic_df.isna().sum()



For age Column The NaN values can be filled with the Median Age

Similarly for Fare Nan Value can be filled with Fare Median value

For Embarked let's fill Nan Values with Southampton,i.e 'S'

C = Cherbourg, Q = Queenstown, S = Southampton

In [ ]:
titanic_df.drop('Cabin',axis=1,inplace=True)
titanic_df['Age']=titanic_df['Age'].fillna(titanic_df['Age'].median())
titanic_df['Embarked']=titanic_df['Embarked'].fillna('S')
titanic_df['Fare']=titanic_df['Fare'].fillna(titanic_df['Fare'].median())

Make sure that the dataset doesn't contain any NaN values

In [ ]:
titanic_df.isna().sum()

Dataset without any NaN values, But it contains both Categorical and numerical data, 
the dataframe can be made completely numerical in the next few steps

In [ ]:
print(titanic_df['Sex'].unique())
print(titanic_df['Embarked'].unique())

Dataframe Sex column contains categorical values, to make it numerical, first it has to be checked what are the unique values in that column and replace those with numerical data, It can be seen that the Sex column has 'Male' and 'Female' as unique values, replacing 'male'=1 and 'female'=0

Similarly for Embarked,The Categorical can be replaced using LabelEncoder

In [ ]:
label_enc=LabelEncoder()

for i in ['Sex','Embarked']:
    titanic_df[i]=label_enc.fit_transform(titanic_df[i])

In [ ]:
titanic_df.describe()

In [ ]:
titanic_df.info()

In [ ]:
titanic_df.corr()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(titanic_df.corr(),annot=True,square=True,linewidth=2)

By the Heatmap we can see that there is a small correlation between parch and SibSp, and Fare and Survived

Checking the total number of Male and Female passenegers who survived the disaster

In [ ]:
sns.countplot(x='Sex',hue='Survived',data=titanic_df)

From the figure it can be said that the Female passangers has higher chance of survival compared with male, and it can also be said that the preference was given to female passangers than male passangers.

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(x='Parch',hue='Survived',data=titanic_df)

From figure we can say that individuals are the one who had higher Survial and Death rate compared with people who had family.

In [ ]:
sns.countplot(x='Pclass',hue='Survived',data=titanic_df)

pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

The survival rate for the passangers in the Lower class was very less, and also we can say that there were more number of people in lower class as compared with the other two classes , the figure is plotted below

In [ ]:
sns.countplot(x='Pclass',data=titanic_df)

Dividing the age of the passangers depening on their age

1]children upto age 10

2]Young from age 10 to 18

3] Adult from age 18 to 50

4] Senior from 50

In [ ]:
for i in range(len(titanic_df)):
    if titanic_df['Age'][i]<=10:
        titanic_df['Age'][i]=1
    if (titanic_df['Age'][i]>10) & (titanic_df['Age'][i]<=18):
            titanic_df['Age'][i]=2
    if (titanic_df['Age'][i]>18) & (titanic_df['Age'][i]<50):
            titanic_df['Age'][i]=3
    if titanic_df['Age'][i]>=50:
            titanic_df['Age'][i]=4

    Lets check what is the survival rate on the age range

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(x='Age',hue='Survived',data=titanic_df)

It can be seen that the large number of people are in the age group from 18 to 50

In [ ]:
plt.hist(titanic_df['Fare'])

In [ ]:
plt.figure(figsize=(10,7))
sns.boxplot(x='Fare',y='Survived',data=titanic_df,orient='h')

It can be seen that there is a small correlation between Fare and survival rate where the surivval rate is quite high, for the ticket charges which is high.

In [ ]:
plt.figure(figsize=(10,7))
sns.catplot(x='Survived',y='Fare',hue='Sex',data=titanic_df,palette='magma',kind='violin')

The survived people Ticket fare is high as compared with Non survived people for both male and female

In [ ]:
sns.countplot(x='Embarked',hue='Survived',data=titanic_df)

There were large number of people who embarked from Southampton due to which there are high number of Deaths and Survival rate,Compared with Queenstown and cherbourg

Training and testing the dataset with LogisticRegression,SVC and RandomForestClassifier

In [ ]:
#X=titanic_df.drop('Survived',axis=1)
#y=titanic_df['Survived']

In [ ]:
#X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)#

In [ ]:
X_train=titanic_df.dropna()
X_train=X_train.drop('Survived',axis=1)

In [ ]:
y_train=train_df['Survived']


In [ ]:
X_test=titanic_df[(titanic_df['Survived']!=0.0) & (titanic_df['Survived']!=1.0)]
X_test=X_test.drop('Survived',axis=1)

In [ ]:
X_test

In [ ]:
X_train

LogisticRegression

In [ ]:
model=LogisticRegression(C=100)
model.fit(X_train,y_train)
test_result=model.predict(X_test)
#roc_auc_score(y_test,test_result)

RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# rand_model=RandomForestClassifier()
# rand_model.fit(X_train,y_train)
# rand_predict=rand_model.predict(X_test)
# roc_auc_score(y_test,rand_predict)

SVC-Support vector Classifier

In [ ]:
# svc_model=SVC()
# svc_model.fit(X_train,y_train)
# svc_predict=svc_model.predict(X_test)
# roc_auc_score(y_test,svc_predict)

Submission

In [ ]:
#predictions=pd.DataFrame(test_result,columns=['predict']).to_csv('predictions.csv')

In [ ]:
submission=pd.DataFrame({'PassengerId':X_test['PassengerId'],'Survived':test_result})

In [ ]:
submission.to_csv('gender_submission.csv',index=False)